In [ ]:
# import pandas as pd
# import numpy as np
# import pathlib 
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import random
# from tqdm import tqdm
# import warnings
# import pandas as pd
# import argparse

# from dataset_paths import *

# warnings.filterwarnings('ignore')

# PROJECT = "combine_ctcandi_ichorcna"
# thres_hypo = 0.3
# thres_hyper = 0.6

# output_version = "20241229";
# outdir = "/media/hieunguyen/HNSD_mini/outdir";

# for mode in ["all", "hypo", "hyper"]:
#     if mode == "all":
#         path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output")
#     elif mode == "hypo":
#         path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output_all_hypo")
#     elif mode == "hyper":
#         path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output_all_hyper")
#     else: 
#         raise ValueError("mode must be either all, hypo or hyper")

#     all_countDMP_results = [item for item in pathlib.Path(path_to_03_output).glob("*/countDMPs.xlsx")]
#     # remove gastric results, not statistically significant
#     all_countDMP_results = [item for item in all_countDMP_results if "Gastric" not in str(item)]

#     testdf = pd.DataFrame()

#     for item in all_countDMP_results:
#         tmpdf = pd.read_excel(item)
#         tmpdf["Label"] = str(item).split("/")[-2]
#         testdf = pd.concat([testdf, tmpdf], axis = 0)
#     print("Number of TMD450 regions that have been tested by TCGA data: {}".format(testdf.shape[0]))
#     if "hyper" not in testdf.columns:
#         testdf["hyper"] = 0
#     if "hypo" not in testdf.columns:
#         testdf["hypo"] = 0

#     testdf["hypo_or_hyper"] = testdf[["hyper", "hypo"]].apply(lambda x: "hyper" if x[0] > x[1] else "hypo", axis = 1)
#     testdf["region_combiname"] = testdf[["Var1", "hypo_or_hyper"]].apply(lambda x: f"{x[0]}_{x[1]}", axis = 1)
#     os.system(f"mkdir -p {path_to_03_output}/pan_cancer")
#     testdf.to_excel(os.path.join(path_to_03_output, "pan_cancer", "countDMPs.xlsx"), index = False)

In [5]:
# import pandas as pd
# df = pd.read_excel("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/REPORT4.xlsx")
# df.to_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/REPORT4.csv")

In [2]:
import pandas as pd 
import pathlib 
import random
import os 
dest = "/media/hieunguyen/GSHD_HN01/raw_data/reads_from_450_regions_with_readname_TMDfull_control"

path = "/media/hieunguyen/GSHD_HN01/raw_data/reads_from_450_regions_with_readname_TMDfull"
metadata  = pd.read_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/REPORT4.csv")
# control_samples = metadata[metadata["Label"] == "Control"]["SampleID"]
# all_files = [item for item in pathlib.Path(path).glob("*.csv")]
# samples = [item.name.split(".")[0] for item in all_files ]
# random.seed(42)
# control_samples = random.sample([item for item in control_samples if item in samples], 900)

# selected_all_files = [item for item in all_files if item.name.split(".")[0] in control_samples]
# for file in selected_all_files:
#     os.system(f"rsync -avh --progress {str(file)} {dest}")

In [6]:
all_files = [item for item in pathlib.Path(dest).glob("*.csv")]
metadata = metadata[metadata["SampleID"].isin([item.name.replace(".sorted.csv", "") for item in all_files])]
metadata.to_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/CONTROL.csv")

In [5]:
metadata

,Unnamed: 0,SampleID,ichorCNA,Label,Run_TM,Run_GW,ECD10K,V3,Recent_unseen1,Realworld,...,Evaluate_Recent_unseen4,--Separate1--,Bismark_GW_BAM,TM_BAM,GW_TM_feature_v0.1,GW_feature_v0.2,TM_feature_v0.2,TM_pipeline_output,Bismark_GW_COV,Preprocessed
619,619,K0AQ62,0.02298,Control,R3890,R3890,0,0,0,0,...,0,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repo/R3890/04_b...,/mnt/BACKUP_NAS05/ECDTM_repo/R3890/align/6-K0A...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,/mnt/DATASM14/hieunho/hieu_data/extract_methyl...,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
620,620,K0AQ63,0.02135,Control,R3890,R3890,0,0,0,0,...,0,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repo/R3890/04_b...,/mnt/BACKUP_NAS05/ECDTM_repo/R3890/align/7-K0A...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,/mnt/DATASM14/hieunho/hieu_data/extract_methyl...,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
629,629,K0AQ73,0.01732,Control,R3904,R3904,0,0,0,0,...,0,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repo/R3904/04_b...,/mnt/BACKUP_NAS05/ECDTM_repo/R3904/align/88-K0...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,/mnt/DATASM14/hieunho/hieu_data/extract_methyl...,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
633,633,K0AQ77,0.03546,Control,R3904,R3904,0,0,0,0,...,0,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repository/04_b...,/mnt/BACKUP_NAS05/ECDTM_repo/R3904/align/92-K0...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,/mnt/DATASM14/hieunho/hieu_data/extract_methyl...,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
642,642,K2AD18,0.03269,Control,R3893,R3893,0,0,0,0,...,0,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repo/R3893/04_b...,/mnt/BACKUP_NAS05/ECDTM_repo/R3893/align/37-K2...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,/mnt/DATASM14/hieunho/hieu_data/extract_methyl...,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,14998,K0AAAK35,NaN,Control,R4775,R4775,0,0,0,0,...,1,NaN,/mnt/GS_NAS05/ECDGW_BIMASK_repo/R4775/04_bisma...,/mnt/BACKUP_NAS05/ECDTM_repo/R4775/align/90-K0...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,NaN,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
15006,15006,K0AAAK50,NaN,Control,R4775,R4775,0,0,0,0,...,1,NaN,/mnt/GS_NAS05/ECDGW_BIMASK_repo/R4775/04_bisma...,/mnt/GS_NAS05/ECDTM_repo/R4775/align/120-K0AAA...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,NaN,/mnt/DATAR09/DATA_HIEUTRAN/preprocessing_pipel...
15010,15010,K0AAAK54,NaN,Control,R4805,R4805,0,0,0,0,...,1,NaN,/mnt/BACKUP_NAS05/ECDGW_BIMASK_repo/R4805/04_b...,/mnt/GS_NAS05/ECDTM_repo/R4805/align/57-K0AAAK...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_project/cancer_clas...,/mnt/DATASM14/hieunho/hieu_data/TM_pipeline_me...,NaN,/datassd/hieunho/preprocessing_pipeline_for_km...
15012,15012,K0AAAK57,NaN,Control,R4805,R4805,0,0,0,0,...,1,NaN,/mnt/GS_NAS05/ECDGW_BIMASK_repo/R4805/04_bisma